# HLS Foundation Model Fine-Tuning Guide

This notebook is designed to offer a guideline for fine-tuning semantic segmentation models using the [HLS Prithvi model](https://github.com/NASA-IMPACT/hls-foundation-os). To run this notebook, you will need a few resources:

1. The HLS Prithvi model weights, which can be found [here](https://huggingface.co/ibm-nasa-geospatial/Prithvi-100M). We recommend downloading this to Google Drive.

2. A segmentation dataset which you can generate by following [this guide](https://colab.research.google.com/drive/1VEJOpyeTJeYSX8EcINe1m7lA-Bi7YoqU?usp=sharing).

3. A configuration file which you can define using [this guide](https://colab.research.google.com/drive/1cOU_JG73dsLmAx4B9yr3tmTvflx0AJ9I?usp=sharing).

Connect to Google Drive to access the config file and training/validation training set.

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive/")

Mounted at /content/gdrive/


###Download all of the dependencies including cloning the HLS Foundation Model github repo.

In [ ]:
from IPython.display import clear_output
import os
!pip install torch==1.11.0+cu115 torchvision==0.12.0+cu115 --extra-index-url https://download.pytorch.org/whl/cu115
!git clone https://github.com/NASA-IMPACT/hls-foundation-os.git
os.chdir('/content/hls-foundation-os')
!pip install -e .
!pip install -U openmim
!mim install mmcv-full==1.6.2 -f https://download.openmmlab.com/mmcv/dist/cu115/torch1.11.0/index.html
!pip install yapf==0.40.1
import torch
clear_output()

###Specify the config file path

In [ ]:
#Define Config file path
config_file_path = "/content/gdrive/MyDrive/HLS foundation guide/hls_config.py"
config_file_path = f'"{config_file_path}"'

###Fine-Tuning
Run the command below to begin fine-tuning. The config file defines all of the important meta-parameters for training, as well as the storage location of the dataset. The log files and model checkpoints will be saved in the experiment file path defined within the config file.

In [ ]:
!mim train mmsegmentation --launcher pytorch {config_file_path}

Using port 27822 for synchronization. 
Training command is /usr/bin/python3 -m torch.distributed.launch --nproc_per_node=1 --master_port=27822 /usr/local/lib/python3.10/dist-packages/mmseg/.mim/tools/train.py /content/gdrive/MyDrive/HLS foundation guide/hls_config.py --launcher pytorch. 
/usr/local/lib/python3.10/dist-packages/torch/distributed/launch.py:178: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  warnings.warn(
2023-10-25 15:12:48,493 - mmseg - INFO - Multi-processing start method is `None`
2023-10-25 15:12:48,498 - mmseg - INFO - OpenCV num_threads is `2
2023-10-25 15:12:48,530 - mmseg - INFO - Environment info:
----------------------------------

###Testing
Evaluating the model can be done with the command below. Here we must define the model checkpoint to evaluate. The performance of the model will improve if we choose to train for more than 5 epochs.

In [ ]:
path_to_checkpoint = "/content/gdrive/MyDrive/HLS foundation guide/HLS_wellpad_finetuning/epoch_5.pth"
path_to_checkpoint = f"'{path_to_checkpoint}'"

!mim test mmsegmentation {config_file_path} --checkpoint {path_to_checkpoint} --eval "mIoU"

Testing command is /usr/bin/python3 /usr/local/lib/python3.10/dist-packages/mmseg/.mim/tools/test.py /content/gdrive/MyDrive/HLS foundation guide/hls_config.py /content/gdrive/MyDrive/HLS foundation guide/HLS_wellpad_finetuning/epoch_5.pth --launcher none --eval mIoU. 
2023-10-24 18:10:14,298 - mmseg - INFO - Multi-processing start method is `None`
2023-10-24 18:10:14,302 - mmseg - INFO - OpenCV num_threads is `2
2023-10-24 18:10:14,306 - mmseg - INFO - Loaded 57 images
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:487: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
load from /content/gdrive/MyDrive/HLS Foundation M

###Generating Inferences
Inferences can be generated easily from a GeoTiff of any size. Ensure that the input Geotiff has the same preproccessing / normalization as the training dataset.

In [ ]:
#Folder location where the input tiff is stored
input_tiff_path = "/content/gdrive/MyDrive/GeoTile Examples/examples/"

#Folder location where inference output will be stored
output_folder_path = "/content/"

input_tiff_path = f"'{input_tiff_path}'"
output_folder_path = f"'{output_folder_path}'"

In [ ]:
!python model_inference.py -config {config_file_path} -ckpt {path_to_checkpoint} -input {input_tiff_path} -output {output_folder_path} -input_type tif -bands 0 1 2 3 4 5


/usr/local/lib/python3.10/dist-packages/mmseg/models/decode_heads/decode_head.py:104: UserWarning: For binary segmentation, we suggest using`out_channels = 1` to define the outputchannels of segmentor, and use `threshold`to convert seg_logist into a predictionapplying a threshold
  warnings.warn('For binary segmentation, we suggest using'
load checkpoint from local path: /content/gdrive/MyDrive/HLS foundation guide/HLS_wellpad_finetuning/epoch_5.pth
Identified images to predict on: 1
Working on Image 0
Running inference...
Output has shape: (4096, 6144)
Saving output...
Inference completed in 189.8 seconds. Output available at: /content/wellpad_sentinel_normalized_pred.tif
